In [14]:
!pip install nilearn
import numpy as np
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import glob
from matplotlib import pyplot as plt
%matplotlib inline
import nilearn as nil
import cv2
from keras.models import load_model

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
print("CorrImage-Normalized-Min: ",np.min(correctedImage_normalized))
print("CorrImage-Normalized-Max: ",np.max(correctedImage_normalized))
print("CorrImage-UnNormalized-Min: ",np.min(correctedImage_unnormalized))
print("CorrImage-UnNormalized-Max: ",np.max(correctedImage_unnormalized))

print("OrigImage-Normalized-Min: ",np.min(image_test_sliced_normalized))
print("OrigImage-Normalized-Max: ",np.max(image_test_sliced_normalized))

print("OrigImage-UnNormalized-Min: ",np.min(image_test_sliced))
print("OrigImage-UnNormalized-Max: ",np.max(image_test_sliced))

print("Predict Bias- Min: ",np.min(predicted_biasField))
print("Predict Bias- Max: ",np.max(predicted_biasField))

In [17]:
N = 64
#get the locations of all the images from the test set
#test_image_list = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/Images/*'))
seven_tesla = 'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/7T-study-475_T1_MP2RAGE_20170608134155_8.bse.nii.gz';

#read one of the test images
image_test = nib.load(seven_tesla)
image_test = image_test.get_fdata()

#Store sliced images in this array
image_test_sliced = []
for k in range(image_test.shape[0]):                                                                                                   #for k in range(a.shape[0]):
  temp = cv2.resize(image_test[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)        #INTER_CUBIC  (type of interpolation)                                     #temp = cv2.resize(a[:,100+k,:], (N, N), interpolation = cv2.INTER_CUBIC) 
  image_test_sliced.append(temp)

for k in range(image_test.shape[1]):
  temp = cv2.resize(image_test[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR) 
  image_test_sliced.append(temp)

for k in range(image_test.shape[2]):
  temp = cv2.resize(image_test[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR) 
  image_test_sliced.append(temp)


image_test_sliced = np.asarray(image_test_sliced)  #converting list to a numpy array (first dimension refers to the #of samples and the second and the third are the 2D image dimension)
image_test_sliced = image_test_sliced.reshape(-1,N,N,1)  #-1 is generally used for an unknown dimension


#Normalize the intensity values (min-max normalization)
max = np.max(image_test_sliced)    #Normalization over all the training images to get the intensity values between 0 and 1 
min = np.min(image_test_sliced)

image_test_sliced_normalized = (image_test_sliced - min) / (max - min)

#Load the model
model = load_model('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Model/final_model.h5')

#Predict the bias field using the model
predicted_biasField = model.predict(image_test_sliced_normalized)

#unnormalize the predicted biasField
predicted_biasField_unnormalized=(predicted_biasField * (max - min) + min)

#predicted_biasField_unnormalized=(predicted_biasField)
                                  
#Don't know why and ridiculous - Comment if the predictions turns out to be wrong
for i in range(len(predicted_biasField_unnormalized)):
    for j in range(N):
      for k in range(N):
        if (image_test_sliced_normalized[i,j,k] == 0): #images array contains data from skull striped images; 
          predicted_biasField_unnormalized[i,j,k] = 0 #this is done to set those pixel values that are outside the brain in the images ground truth to 0


#Generate corrected Image
correctedImage_normalized = np.empty([752, 64, 64,1])
for i in range(0, len(image_test_sliced_normalized)):
  for j in range(0, N):
    for k in range(0, N):
      if (predicted_biasField_unnormalized[i,j,k] == 0):
        correctedImage_normalized[i,j,k] = image_test_sliced_normalized[i,j,k]
      else:
        correctedImage_normalized[i,j,k] = image_test_sliced_normalized[i,j,k]/predicted_biasField_unnormalized[i,j,k]

#Unnormalize the corrected image
#correctedImage_unnormalized = (correctedImage_normalized*(max - min)) + min
correctedImage_unnormalized = (correctedImage_normalized)
#Reshape the corrected Image
correctedImage = np.empty([192,280,280])

for idx in range(0,192):
      img = correctedImage_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280, 280), interpolation=cv2.INTER_CUBIC)
      correctedImage[idx,:,:]=img_resized

for idx in range(192,472):
      img = correctedImage_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280, 192), interpolation=cv2.INTER_CUBIC)
      correctedImage[:,idx-192,:]=img_resized

for idx in range(472, 752):
      img = correctedImage_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280,192), interpolation=cv2.INTER_CUBIC)
      correctedImage[:,:,idx-472]=img_resized



#Reshape the predicted biasField
final_predicted_biasField = np.empty([192,280,280])

for idx in range(0,192):
      img = predicted_biasField_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280, 280), interpolation=cv2.INTER_CUBIC)
      final_predicted_biasField[idx,:,:]=img_resized

for idx in range(192,472):
      img = predicted_biasField_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280, 192), interpolation=cv2.INTER_CUBIC)
      final_predicted_biasField[:,idx-192,:]=img_resized

for idx in range(472, 752):
      img = predicted_biasField_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (280,192), interpolation=cv2.INTER_CUBIC)
      final_predicted_biasField[:,:,idx-472]=img_resized



In [19]:
print(correctedImage[120][167][145])
print(final_predicted_biasField[120][167][145])
print(image_test[120][167][145])


print("Min-Max Corrected MRI:",np.min(correctedImage), np.max(correctedImage))
print("Min-Max Predicted Bias Field:",np.min(final_predicted_biasField), np.max(final_predicted_biasField))
print("Min-Max Input MRI:",np.min(image_test), np.max(image_test))


0.001079408280953612
262.95074462890625
299.0
Min-Max Corrected MRI: -0.3286949138017656 2.956548618608426
Min-Max Predicted Bias Field: -190.10533142089844 995.3699340820312
Min-Max Input MRI: 0.0 1136.0


In [20]:
#Save the output imagesbr

#Corrected MRI images
correctedImage = nil.image.new_img_like(nib.load(seven_tesla),correctedImage,copy_header=False)
nib.save(correctedImage,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/7T/corrected_mri_test.nii.gz')

#Estimated bias field
final_predicted_biasField = nil.image.new_img_like(nib.load(seven_tesla),final_predicted_biasField,copy_header=False)
nib.save(final_predicted_biasField,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/7T/reconstructed_bias_test.nii.gz')

#Estimated bias field
image_test = nil.image.new_img_like(nib.load(seven_tesla),image_test,copy_header=False)
nib.save(image_test,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/7T/input_image_test.nii.gz')

In [ ]:
#Jaccard Coefficient (IOU)
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

#Step 1: Read all the labels
ground_truth = nib.load('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/0/tissue_classification/corrected_train-1.pvc.label.nii.gz');
without_BFC = nib.load('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/0/tissue_classification/input_image_test0.pvc.label.nii.gz');
with_BFC_BrainSuit = nib.load('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/0/tissue_classification/input_image_test0.bfc.pvc.label.nii.gz');
with_BFC_autoencoder = nib.load('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/0/tissue_classification/corrected_mri_test0.pvc.label.nii.gz');
ground_truth = ground_truth.get_fdata()
without_BFC = without_BFC.get_fdata()
with_BFC_BrainSuit = with_BFC_BrainSuit.get_fdata()
with_BFC_autoencoder = with_BFC_autoencoder.get_fdata()

ground_truth = ground_truth.flatten()
without_BFC = without_BFC.flatten()
with_BFC_BrainSuit = with_BFC_BrainSuit.flatten()
with_BFC_autoencoder = with_BFC_autoencoder.flatten()

labels_include = [1, 2, 3, 4, 5, 6]




N = 64;

#Step 2: Find Jaccard Coefficient (IoU)
a1 = jaccard_score(ground_truth, without_BFC, average='weighted', labels=labels_include)
print("Ground Truth vs Without BFC: ", a1)

a2 = jaccard_score(ground_truth, with_BFC_BrainSuit, average='weighted', labels=labels_include)
print("Ground Truth vs With BFC from BrainSuit: ", a2)

a3 = jaccard_score(ground_truth, with_BFC_autoencoder, average='weighted', labels=labels_include)
print("Ground Truth vs With BFC from Autoencoder: ", a3)



#Step 3: Find F1-Score (Dice Coefficient)
f1 = f1_score(ground_truth, without_BFC, average='weighted', labels=labels_include)
print("Ground Truth vs Without BFC - F1: ", f1)

f2 = f1_score(ground_truth, with_BFC_BrainSuit, average='weighted', labels=labels_include)
print("Ground Truth vs With BFC from BrainSuit - F1: ", f2)

f3 = f1_score(ground_truth, with_BFC_autoencoder, average='weighted', labels=labels_include)
print("Ground Truth vs With BFC from Autoencoder - F1: ", f3)


Ground Truth vs Without BFC:  0.15846849530367968
Ground Truth vs With BFC from BrainSuit:  0.31721006214774355
Ground Truth vs With BFC from Autoencoder:  0.3805371521612907
Ground Truth vs Without BFC - F1:  0.26132827271622283
Ground Truth vs With BFC from BrainSuit - F1:  0.46406881496173613
Ground Truth vs With BFC from Autoencoder - F1:  0.5383244281579139


In [3]:
#Make the Ground Truth Bias correct.
Labels = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/Labels/*'))
test_image_list = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/Images/*'))

N = 64
#read one of the labels
bias_field = nib.load(Labels[0])
bias_field = bias_field.get_fdata()

#read one of the test images
image_test = nib.load(test_image_list[0])
image_test = image_test.get_fdata()

#Store sliced images in this array
image_test_sliced = []
for k in range(image_test.shape[0]):                                                                                                   #for k in range(a.shape[0]):
  temp = cv2.resize(image_test[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)        #INTER_CUBIC  (type of interpolation)                                     #temp = cv2.resize(a[:,100+k,:], (N, N), interpolation = cv2.INTER_CUBIC) 
  image_test_sliced.append(temp)

for k in range(image_test.shape[1]):
  temp = cv2.resize(image_test[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR) 
  image_test_sliced.append(temp)

for k in range(image_test.shape[2]):
  temp = cv2.resize(image_test[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR) 
  image_test_sliced.append(temp)

#Slice the bias field
bias_field_sliced = []
for k in range(bias_field.shape[0]):                                                                                                   #for k in range(a.shape[0]):
  temp = cv2.resize(bias_field[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)        #INTER_CUBIC  (type of interpolation)                                     #temp = cv2.resize(a[:,100+k,:], (N, N), interpolation = cv2.INTER_CUBIC) 
  bias_field_sliced.append(temp)

for k in range(bias_field.shape[1]):
  temp = cv2.resize(bias_field[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR) 
  bias_field_sliced.append(temp)

for k in range(bias_field.shape[2]):
  temp = cv2.resize(bias_field[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR) 
  bias_field_sliced.append(temp)

max = np.max(image_test_sliced)    #Normalization over all the training images to get the intensity values between 0 and 1 
min = np.min(image_test_sliced)

image_test_sliced_normalized = (image_test_sliced - min) / (max - min)
bias_field_sliced_normalized = (bias_field_sliced-min) / (max - min)

for i in range(len(bias_field_sliced_normalized)):
    for j in range(N):
      for k in range(N):
        if (image_test_sliced_normalized[i,j,k] == 0): #images array contains data from skull striped images; 
          bias_field_sliced_normalized[i,j,k] = 0 #this is done to set those pixel values that are outside the brain in the images ground truth to 0


#Reshape the bias field
ground_truth_biasfield = np.empty([260,311,260])

for idx in range(0,260):
      img = bias_field_sliced_normalized[idx, :, :]
      img_resized = cv2.resize(img, (260, 311), interpolation=cv2.INTER_CUBIC)
      ground_truth_biasfield[idx,:,:]=img_resized

for idx in range(260,571):
      img = bias_field_sliced_normalized[idx, :, :]
      img_resized = cv2.resize(img, (260, 260), interpolation=cv2.INTER_CUBIC)
      ground_truth_biasfield[:,idx-260,:]=img_resized

for idx in range(571, 831):
      img = bias_field_sliced_normalized[idx, :, :]
      img_resized = cv2.resize(img, (311,260), interpolation=cv2.INTER_CUBIC)
      ground_truth_biasfield[:,:,idx-571]=img_resized


#Original bias field
final_predicted_biasField = nil.image.new_img_like(nib.load(test_image_list[0]),ground_truth_biasfield,copy_header=True)
nib.save(final_predicted_biasField,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Results/0/ground_truth_biasfield_1.nii.gz')

